In [1]:
from constants import *
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import re
import pickle
from gensim.models import Word2Vec, Doc2Vec
from word2vec import EpochLogger, EpochSaver

pd.options.display.max_rows = 500

-----

### re-implementation of 'pruned_documents.py'

In [6]:
path = 'NETL/additional_support_files'
#file = 'short_label_documents_de_NETL_implementation'
file = 'short_label_documents_en'
filepath = join(path, file)

In [2]:
with open(filepath, 'rb') as fp:
    lst = pickle.load(fp, encoding='utf8')
    netl = pd.Series(lst)
len(lst), lst[4242002-1]

(4242002, 'random_ep_(number_1)')

In [10]:
dewiki_phrases = 'dewiki_phrases.pickle'
dewiki_phrases_joined = 'dewiki_phrases_joined.pickle'
dewiki_phrases_lemmatized = 'dewiki_phrases_lemmatized.pickle'

dp = pd.read_pickle(join(ETL_PATH, dewiki_phrases))
dpj = pd.read_pickle(join(ETL_PATH, dewiki_phrases_joined))
dpl = pd.read_pickle(join(ETL_PATH, dewiki_phrases_lemmatized))

In [11]:
dpl

,sent_idx,text,token,doc_len,title_len
hash,,,,,
8952056961092092653,1,Alan_Smithee,Alan_Smithee,747,2
598046625986755870,44,Actinium,Actinium,866,1
8442369265370766621,98,Ang_Lee,ANG_Lee,1632,2
-5325279570187525080,181,Anschluss,Anschluss,413,1
5107548614255273253,199,Aussagenlogik,Aussagenlogik,7211,1
-6810310479569543740,599,Anthony_Minghella,Anthony_Minghella,637,2
-291419119128528545,636,US-amerikanischer_Film,us-amerikanisch_Film,2992,2
-686601136003585762,767,Vorsätze_für_Maßeinheiten,Vorsatz_für_Maßeinheit,992,3
-928617659304474122,829,Abkürzungen/Gesetze_und_Recht,Abkürzungen/Gesetz_und_Recht,167,3


In [12]:
title_length = 5 # filter < 5
doc_length = 40 # filter > 40

In [8]:
# without lemmatization

output_filename = 'short_label_documents_de_NEW_implementation'
goodids = pd.read_pickle(join(ETL_PATH, 'dewiki_good_ids.pickle'))
df = pd.read_pickle(join(ETL_PATH, 'dewiki_new.pickle'))
df = df[[TITLE, TEXT]]
df = df.reset_index(drop=False)
df = df[df.hash_nlp.isin(goodids.index)]
get_len = lambda x: len(x.split())
df.loc[:, 'title_len'] = df.title.map(get_len)
df.loc[:, 'text_len'] = df.text.map(get_len)
df = df.query(f'(title_len < {title_length}) and (text_len > {doc_length})')

titles = df.title.str.replace(' ', '_')
lst_new = titles.tolist()


output_file = 'short_label_documents_de_NEW_implementation'
outpath = join(path, output_file)
print("Writing labels to pickle file", outpath)
with open(outpath, 'wb') as fp:
    pickle.dump(lst_new, fp)
    
lst_new

['Alan_Smithee',
 'Actinium',
 'Ang_Lee',
 'Anschluss',
 'Aussagenlogik',
 'Anthony_Minghella',
 'US-amerikanischer_Film',
 'Vorsätze_für_Maßeinheiten',
 'Abkürzungen/Gesetze_und_Recht',
 'ISO_4217',
 'Achsensprung',
 'Alfred_Hitchcock',
 'Auteur-Theorie',
 'Aki_Kaurismäki',
 'Anime',
 'Actionfilm',
 'Al_Pacino',
 'Alkohole',
 'The_Visitors',
 'Aluminium',
 'Antimon',
 'Argon',
 'Arsen',
 'Astat',
 'Alkalimetalle',
 'Actinoide',
 'Americium',
 'Atom',
 'Arzt',
 'Anthropologie',
 'Alexander_der_Große',
 'Antike',
 'Anthony_Hope',
 'Ångström',
 'Ampere',
 'Acre',
 'Ar',
 'Arbeit',
 'Atomare_Masseneinheit',
 'Anglizismus',
 'Astronom',
 'Alan_Turing',
 'Archäologie',
 'Außenbandruptur_des_oberen_Sprunggelenkes',
 'Alphabet',
 'Arthur_Harris',
 'Astronomie',
 'Angelina_Jolie',
 'Archimedes',
 'Aristoteles',
 'Abraham_Lincoln',
 'Altweibersommer',
 'Astana',
 'Angela_Merkel',
 'Alicia_Silverstone',
 'Al-Biruni',
 'Ankara',
 'Atheismus',
 'Anna_Seghers',
 'Atmosphäre',
 'Arvo_Pärt',
 'Afghan

In [15]:
# creating a list containing original and lemmatized phrases

x = dpl.query('doc_len > @doc_length and title_len < @title_length')
lst_lemma = x.text.append(x.token).tolist()
lst_lemma = sorted(set(lst_lemma))

output_file = 'short_label_documents_de_lemmatized'
outpath = join(path, output_file)
print("Writing labels to pickle file", outpath)
with open(outpath, 'wb') as fp:
    pickle.dump(lst_lemma, fp)
    
len(lst_lemma), lst_lemma

Writing labels to pickle file NETL/additional_support_files/short_label_documents_de_lemmatized


(1778215,
 ['!_!_!',
  '!_Distain',
  '!_distain',
  '$_50SAT',
  '&_RQ',
  '&_Radieschen',
  '(_)',
  '(_.)p(_..._)nin',
  '(_030_)_Magazin',
  '(_1,5-Cyclooctadien)(1,3,5-cyclooctatrien)ruthenium',
  '(_100000_)_Astronautica',
  '(_10000_)_Myriostos',
  '(_10001_)_Palermo',
  '(_100027_)_Hannaharendt',
  '(_100033_)_Taizé',
  '(_10007_)_Malytheatre',
  '(_1000_)_Piazzia',
  '(_10010_)_Rudruna',
  '(_100133_)_Demosthenes',
  '(_10015_)_Valenlebedev',
  '(_1001_)_Gaussia',
  '(_100267_)_JAXA',
  '(_100268_)_Rosenthal',
  '(_1002_)_Olbersia',
  '(_10030_)_Philkeenan',
  '(_10038_)_Tanaro',
  '(_1003_)_Lilofee',
  '(_10041_)_Parkinson',
  '(_10044_)_Squyres',
  '(_100483_)_NAOJ',
  '(_1004_)_Belopolskya',
  '(_10055_)_Silcher',
  '(_10057_)_L’Obel',
  '(_1005_)_Arago',
  '(_100604_)_Lundy',
  '(_10067_)_Bertuch',
  '(_10068_)_Dodoens',
  '(_10069_)_Fontenelle',
  '(_1006_)_Lagrangea',
  '(_10071_)_Paraguay',
  '(_10072_)_Uruguay',
  '(_10079_)_Meunier',
  '(_1007_)_Pawlowia',
  '(_10088_

-----

### re-implementation of 'get_indices.py'

In [ ]:
doc2vec_indices_file_en = join(path, 'doc2vec_indices_en')
word2vec_indices_file_en = join(path, 'word2vec_indices_en')

# creating output indices file
with open(doc2vec_indices_file_en, 'rb') as fp:
    doc_indices_en = pickle.load(fp)
with open(word2vec_indices_file_en, 'rb') as fp:
    word_indices_en = pickle.load(fp)

word_indices_en

In [2]:
d2vpath = 'NETL/training/trained_models/doc2vec_lemma/wiki_lemma.d2v'
w2vpath = 'NETL/training/trained_models/w2v_lemma/w2v_lemma'
print("Doc2Vec loading", d2vpath)
d2v = Doc2Vec.load(d2vpath)
print("Word2Vec loading", w2vpath)
w2v = Word2Vec.load(w2vpath)
print("models loaded")

Doc2Vec loading NETL/training/trained_models/doc2vec_lemma/wiki_lemma.d2v
Word2Vec loading NETL/training/trained_models/w2v_lemma/w2v_lemma
models loaded


In [3]:
d2v.wv.vocab

{'Alan_Smithee': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d208>,
 'stehen': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d2e8>,
 'als': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d358>,
 'Pseudonym': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d390>,
 'für': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d3c8>,
 'ein': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d438>,
 'fiktiv': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d4a8>,
 'Regisseur': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d4e0>,
 'der': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d550>,
 'Film': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d5c0>,
 'verantworten': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d5f8>,
 'bei': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d668>,
 'eigentlich': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d6a0>,
 'seinen': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d710>,
 'Name': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d780>,
 'nicht': <gensim.mode

In [4]:
w2v.wv.vocab

{'Alan_Smithee': <gensim.models.keyedvectors.Vocab at 0x7f4bf5430f28>,
 'stehen': <gensim.models.keyedvectors.Vocab at 0x7f4bf5430c50>,
 'als': <gensim.models.keyedvectors.Vocab at 0x7f4bf5430a58>,
 'Pseudonym': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d1d0>,
 'für': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d198>,
 'ein': <gensim.models.keyedvectors.Vocab at 0x7f4bf543d278>,
 'fiktiv': <gensim.models.keyedvectors.Vocab at 0x7f4be80825f8>,
 'Regisseur': <gensim.models.keyedvectors.Vocab at 0x7f4be8082588>,
 'der': <gensim.models.keyedvectors.Vocab at 0x7f4be80826a0>,
 'Film': <gensim.models.keyedvectors.Vocab at 0x7f4be8082710>,
 'verantworten': <gensim.models.keyedvectors.Vocab at 0x7f4be8082748>,
 'bei': <gensim.models.keyedvectors.Vocab at 0x7f4be80827b8>,
 'eigentlich': <gensim.models.keyedvectors.Vocab at 0x7f4be80827f0>,
 'seinen': <gensim.models.keyedvectors.Vocab at 0x7f4be8082860>,
 'Name': <gensim.models.keyedvectors.Vocab at 0x7f4be80828d0>,
 'nicht': <gensim.mode

In [16]:
in_file = 'short_label_documents_de_lemmatized'
inpath = join(path, in_file)
with open(inpath, 'rb') as fp:
    labels_de = pickle.load(fp)

labels_de

['!_!_!',
 '!_Distain',
 '!_distain',
 '$_50SAT',
 '&_RQ',
 '&_Radieschen',
 '(_)',
 '(_.)p(_..._)nin',
 '(_030_)_Magazin',
 '(_1,5-Cyclooctadien)(1,3,5-cyclooctatrien)ruthenium',
 '(_100000_)_Astronautica',
 '(_10000_)_Myriostos',
 '(_10001_)_Palermo',
 '(_100027_)_Hannaharendt',
 '(_100033_)_Taizé',
 '(_10007_)_Malytheatre',
 '(_1000_)_Piazzia',
 '(_10010_)_Rudruna',
 '(_100133_)_Demosthenes',
 '(_10015_)_Valenlebedev',
 '(_1001_)_Gaussia',
 '(_100267_)_JAXA',
 '(_100268_)_Rosenthal',
 '(_1002_)_Olbersia',
 '(_10030_)_Philkeenan',
 '(_10038_)_Tanaro',
 '(_1003_)_Lilofee',
 '(_10041_)_Parkinson',
 '(_10044_)_Squyres',
 '(_100483_)_NAOJ',
 '(_1004_)_Belopolskya',
 '(_10055_)_Silcher',
 '(_10057_)_L’Obel',
 '(_1005_)_Arago',
 '(_100604_)_Lundy',
 '(_10067_)_Bertuch',
 '(_10068_)_Dodoens',
 '(_10069_)_Fontenelle',
 '(_1006_)_Lagrangea',
 '(_10071_)_Paraguay',
 '(_10072_)_Uruguay',
 '(_10079_)_Meunier',
 '(_1007_)_Pawlowia',
 '(_10088_)_Digne',
 '(_10089_)_Turgot',
 '(_10093_)_Diesel',
 '

In [17]:
doc_indices_de = []
word_indices_de = []

dout = []
wout = []
for label in labels_de[:]:
    try:
        idx = d2v.wv.vocab[label].index
        doc_indices_de.append(idx)
    except:
        dout.append(label)
    try:
        idx = w2v.wv.vocab[label].index
        word_indices_de.append(idx)
    except:
        wout.append(label)
        # print(f'{label} not in w2v vocab')

doc_indices_de

[461289,
 347430,
 61639,
 255435,
 253603,
 270891,
 474266,
 640217,
 628085,
 91476,
 450811,
 82255,
 106118,
 286865,
 424665,
 654934,
 382189,
 487412,
 190939,
 310711,
 227443,
 400529,
 488501,
 299539,
 186114,
 549809,
 300587,
 396203,
 59039,
 10703,
 347317,
 504333,
 22511,
 157957,
 599506,
 78,
 567946,
 264832,
 708396,
 395381,
 489330,
 272180,
 405396,
 588924,
 136959,
 641985,
 459300,
 405804,
 679751,
 652071,
 436124,
 684695,
 526828,
 512962,
 399032,
 636965,
 277924,
 583055,
 693773,
 690492,
 344116,
 448811,
 217,
 43170,
 700661,
 435650,
 603914,
 682635,
 317149,
 347450,
 308243,
 156629,
 223525,
 207831,
 646724,
 44277,
 55830,
 57328,
 7726,
 328582,
 39915,
 53701,
 47851,
 45941,
 45151,
 51079,
 40538,
 59159,
 280399,
 32170,
 42577,
 47740,
 62952,
 342108,
 58324,
 6570,
 47293,
 329081,
 8575,
 290982,
 50970,
 45085,
 177265,
 483865,
 305860,
 394856,
 223190,
 645845,
 348531,
 337,
 8515,
 42859,
 305615,
 25965,
 221822,
 575130,
 6

In [18]:
# most of the labels are OOV, probably due to min_count=20 (that parametrization is from the NETL implementation)
len(doc_indices_de), len(dout), len(word_indices_de), len(wout)

(291511, 1486704, 291511, 1486704)

In [19]:
doc2vec_indices_file_de = join(path, 'doc2vec_indices_de')
word2vec_indices_file_de = join(path, 'word2vec_indices_de')

# creating output indices file
with open(doc2vec_indices_file_de, 'wb') as fp:
    pickle.dump(doc_indices_de, fp)
with open(word2vec_indices_file_de, 'wb') as fp:
    pickle.dump(word_indices_de, fp)

-----
### Pagerank titles -> remove categories from title phrases

In [ ]:
path = '../../master_cloud/external_code/NETL-Automatic-Topic-Labelling-/model_run/support_files/de/'
df = pd.read_csv(join(path, 'pagerank-titles-sorted_de.txt'), sep='\t', header=None, keep_default_na=False)

def process_titles(title):
    title = title.split('(')[0].strip('_')
    return title

df[1] = df[1].map(process_titles)
df.to_csv(join(path, 'pagerank-titles-sorted_de_categories_removed.txt'), sep='\t', header=None, index=None)